# 1. Импортирование библиотек

In [1]:
import pandas as pd  # Импорт библиотеки pandas для работы с данными
import torch  # Импорт библиотеки PyTorch для работы с нейронными сетями

from torch.utils.data import Dataset  # Импорт класса Dataset из PyTorch для создания пользовательского датасета
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments  # Импорт необходимых классов из библиотеки transformers
from sklearn.model_selection import train_test_split  # Импорт функции для разделения данных на тренировочный и тестовый наборы

C:\Users\orala\Documents\Anaconda\envs\gpu_tf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Загрузка набора данных

In [2]:
file_path = 'dataset/Симптомы_ответы.csv'  # Путь к файлу с данными
data = pd.read_csv(file_path)  # Загрузка данных в DataFrame

### 2.1. Объединение колонок жалоб и ключевых слов для создания контекста

In [3]:
data['combined'] = data['Complaints'] + " " + data['Keywords']  

# 3. Подготовка датасета

### 3.1. Кодирование меток

In [4]:
label_encoder = {label: idx for idx, label in enumerate(data['Answer'].unique())}  # Создание словаря для кодирования уникальных ответов в числовые метки
data['label'] = data['Answer'].map(label_encoder)  # Применение кодирования к колонке ответов

### 3.2. Разделение данных на тренировочный и тестовый наборы

In [5]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)  # Разделение данных с использованием функции train_test_split

### 3.3. Создание пользовательского класса Dataset

In [7]:
# Создание пользовательского класса Dataset
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_token_len=512):
        self.tokenizer = tokenizer  # Инициализация токенизатора
        self.data = data  # Инициализация данных
        self.max_token_len = max_token_len  # Максимальная длина токенов

    def __len__(self):
        return len(self.data)  # Возвращает количество элементов в датасете

    def __getitem__(self, idx):
        item = self.data.iloc[idx]  # Получение элемента по индексу
        encoding = self.tokenizer.encode_plus(
            item['combined'],  # Текст для токенизации
            add_special_tokens=True,  # Добавление специальных токенов (CLS, SEP)
            max_length=self.max_token_len,  # Максимальная длина токенов
            padding="max_length",  # Дополнение до максимальной длины
            truncation=True,  # Усечение текста до максимальной длины
            return_attention_mask=True,  # Возвращение маски внимания
            return_tensors='pt',  # Возвращение тензоров PyTorch
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),  # Токенизированные ID
            'attention_mask': encoding['attention_mask'].flatten(),  # Маска внимания
            'labels': torch.tensor(item['label'], dtype=torch.long)  # Метки в формате тензоров
        }

### 3.4. Инициализация токенизатора и датасетов

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Загрузка предобученного токенизатора BERT
train_dataset = CustomDataset(train_data, tokenizer)  # Создание тренировочного датасета
test_dataset = CustomDataset(test_data, tokenizer)  # Создание тестового датасета

# 4. Инициализация модели

In [8]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder))  # Загрузка предобученной модели BERT для классификации последовательностей

# 5. Аргументы тренировки и инициализация Trainer

In [10]:
training_args = TrainingArguments(
    output_dir='./results',  # Директория для сохранения результатов
    num_train_epochs=5,  # Количество эпох
    per_device_train_batch_size=4,  # Размер батча для тренировки
    per_device_eval_batch_size=4,  # Размер батча для оценки
    warmup_steps=500,  # Количество шагов для разогрева
    weight_decay=0.01,  # Коэффициент затухания весов
    logging_dir='./logs',  # Директория для сохранения логов
    logging_steps=10,  # Частота логирования
    evaluation_strategy="epoch",  # Стратегия оценки на каждом эпохе
    save_strategy="epoch",  # Стратегия сохранения на каждом эпохе
    load_best_model_at_end=True,  # Загрузка лучшей модели в конце
)

trainer = Trainer(
    model=model,  # Модель для тренировки
    args=training_args,  # Аргументы тренировки
    train_dataset=train_dataset,  # Тренировочный датасет
    eval_dataset=test_dataset  # Тестовый датасет
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 6. Тренировка модели 

In [11]:
trainer.train()

Step,Training Loss
10,3.468400


TrainOutput(global_step=15, training_loss=3.4565185546875, metrics={'train_runtime': 337.8886, 'train_samples_per_second': 0.355, 'train_steps_per_second': 0.044, 'total_flos': 31581264199680.0, 'train_loss': 3.4565185546875, 'epoch': 5.0})

### 6.1. Сохранить модель и токенизатор

In [ ]:
model_save_path = './trained_model'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

# 7. Оценка модели

In [12]:
results = trainer.evaluate()

print("Evaluation results:", results)

Evaluation results: {'eval_loss': 3.3982083797454834, 'eval_runtime': 7.4696, 'eval_samples_per_second': 0.803, 'eval_steps_per_second': 0.134, 'epoch': 5.0}


# 10. Загрузка модели

In [8]:
from transformers import BertTokenizer, BertForSequenceClassification

model_save_path = './trained_model'
loaded_model = BertForSequenceClassification.from_pretrained(model_save_path)
loaded_tokenizer = BertTokenizer.from_pretrained(model_save_path)

print("Model and tokenizer loaded successfully")

def process_user_input(input_text):
    inputs = loaded_tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")  # Токенизация пользовательского ввода
    return inputs

def predict_condition(inputs):
    with torch.no_grad():  # Отключение вычисления градиентов для режима оценки
        predictions = loaded_model(**inputs)  # Получение предсказаний от модели
    return torch.argmax(predictions.logits)  # Возвращение индекса класса с наибольшей вероятностью

def index_to_condition(index):
    condition = {idx: label for label, idx in label_encoder.items()}  # Обратное кодирование меток в текстовые ответы
    return condition[index.item()]

def get_medical_advice(input_text):
    processed_input = process_user_input(input_text)  # Обработка пользовательского ввода
    condition_index = predict_condition(processed_input)  # Получение предсказания модели
    condition = index_to_condition(condition_index)  # Преобразование индекса в текстовый ответ
    return condition

while True:
    user_input = input("Введите вашу жалобу (или 'Стоп' для завершения): ")  # Ввод пользователя
    if user_input.lower() == "стоп":  # Проверка на завершение
        break
    advice = get_medical_advice(user_input)  # Получение медицинского совета
    print("Рекомендации:", advice)  # Печать предсказания

Model and tokenizer loaded successfully
Введите вашу жалобу (или 'Стоп' для завершения): кашель
Рекомендации: Тщательная очистка и дезинфекция раны спиртом или перекисью водорода. Использование стерильных повязок для предотвращения инфекции, к примеру бинт. Наблюдение за раной и обращение к врачу при признаках инфекции или замедленного заживления.
Введите вашу жалобу (или 'Стоп' для завершения): боль в спине
Рекомендации: Прекращение курения и других вредных привычек. Применение медикаментозной терапии для улучшения кровообращения. Регулярные посещения кардиолога для контроля состояния.
Введите вашу жалобу (или 'Стоп' для завершения): зубная боль
Рекомендации: Тщательная очистка и дезинфекция раны спиртом или перекисью водорода. Использование стерильных повязок для предотвращения инфекции, к примеру бинт. Наблюдение за раной и обращение к врачу при признаках инфекции или замедленного заживления.
Введите вашу жалобу (или 'Стоп' для завершения): боли в животе
Рекомендации: УЗИ или КТ для

# 8. Функции для предсказания

In [ ]:
def process_user_input(input_text):
    inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")  # Токенизация пользовательского ввода
    return inputs

def predict_condition(inputs):
    with torch.no_grad():  # Отключение вычисления градиентов для режима оценки
        predictions = model(**inputs)  # Получение предсказаний от модели
    return torch.argmax(predictions.logits)  # Возвращение индекса класса с наибольшей вероятностью

def index_to_condition(index):
    # Обратное кодирование меток в текстовые ответы
    condition = {idx: label for label, idx in label_encoder.items()}
    return condition[index.item()]

def get_medical_advice(input_text):
    processed_input = process_user_input(input_text)  # Обработка пользовательского ввода
    condition_index = predict_condition(processed_input)  # Получение предсказания модели
    condition = index_to_condition(condition_index)  # Преобразование индекса в текстовый ответ
    return condition

# 9. Непрерывный ввод пользователя до команды "Стоп"

In [ ]:
while True:
    user_input = input("Введите вашу жалобу (или 'Стоп' для завершения): ")  # Ввод пользователя
    if user_input.lower() == "стоп":  # Проверка на завершение
        break
    advice = get_medical_advice(user_input)  # Получение медицинского совета
    print("Возможное состояние:", advice)  # Печать предсказания

# Изначальный тест кода

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Load and prepare dataset
file_path = 'dataset/Симптомы_ответы.csv'
data = pd.read_csv(file_path)
data['combined'] = data['Complaints'] + " " + data['Keywords']

# Label encoding
label_encoder = {label: idx for idx, label in enumerate(data['Answer'].unique())}
data['label'] = data['Answer'].map(label_encoder)

# Train-test split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_token_len=512):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            item['combined'],
            add_special_tokens=True,
            max_length=self.max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(item['label'], dtype=torch.long)
        }

# Initialize tokenizer and datasets
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = CustomDataset(train_data, tokenizer)
test_dataset = CustomDataset(test_data, tokenizer)

# Initialize model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder))

# Training arguments and trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,  # Increase the number of epochs
    per_device_train_batch_size=4,  # Decrease batch size
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,  # Load the best model at the end
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Prediction functions
def process_user_input(input_text):
    inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")
    return inputs

def predict_condition(inputs):
    with torch.no_grad():
        predictions = model(**inputs)
    return torch.argmax(predictions.logits)

def index_to_condition(index):
    # Reverse the label encoding to get the answer
    condition = {idx: label for label, idx in label_encoder.items()}
    return condition[index.item()]

def get_medical_advice(input_text):
    processed_input = process_user_input(input_text)
    condition_index = predict_condition(processed_input)
    condition = index_to_condition(condition_index)
    return condition

C:\Users\orala\Documents\Anaconda\envs\gpu_tf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,6.153500,6.133895
2,5.916100,6.260179
3,5.627500,6.080957
4,5.260800,6.070578
5,5.000200,6.063757


Evaluation results: {'eval_loss': 6.063757419586182, 'eval_runtime': 94.5742, 'eval_samples_per_second': 2.189, 'eval_steps_per_second': 0.55, 'epoch': 5.0}


In [2]:
results = trainer.evaluate()

print("Evaluation results:", results)

Evaluation results: {'eval_loss': 6.063757419586182, 'eval_runtime': 94.0747, 'eval_samples_per_second': 2.2, 'eval_steps_per_second': 0.553, 'epoch': 5.0}


In [4]:
# Save the trained model and tokenizer
model_save_path = './trained_model'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to ./trained_model


In [5]:
# Continuous user input until "Стоп"
while True:
    user_input = input("Введите вашу жалобу (или 'Стоп' для завершения): ")
    if user_input.lower() == "стоп":
        break
    advice = get_medical_advice(user_input)
    print("Рекомендации:", advice)

Введите вашу жалобу (или 'Стоп' для завершения): cough
Рекомендации: Maintain hygiene and use antiseptics. Use of antibiotics or antiviral drugs as prescribed by a doctor. It is mandatory to consult a doctor for diagnosis and prescription of adequate treatment.
Введите вашу жалобу (или 'Стоп' для завершения): fever
Рекомендации: Maintain hygiene and use antiseptics. Use of antibiotics or antiviral drugs as prescribed by a doctor. It is mandatory to consult a doctor for diagnosis and prescription of adequate treatment.
Введите вашу жалобу (или 'Стоп' для завершения): Стоп


### Translate to English

In [4]:
import pandas as pd  # Импорт библиотеки pandas для работы с данными
from deep_translator import GoogleTranslator  # Импорт класса GoogleTranslator из библиотеки deep_translator для перевода текста

# Загрузка датасета
file_path = 'dataset/Симптомы_ответы.csv'  # Путь к файлу с данными
data = pd.read_csv(file_path)  # Загрузка данных в DataFrame

# Инициализация переводчика
translator = GoogleTranslator(source='ru', target='en')  # Инициализация переводчика с исходным языком русский и целевым английский

# Функция для перевода текста
def translate_text(text):
    try:
        return translator.translate(text)  # Пытаемся перевести текст
    except Exception as e:
        return text  # Возвращаем оригинальный текст в случае ошибки перевода

# Перевод датасета на английский язык
data_translated = data.copy()  # Создаем копию оригинального датасета
data_translated['Complaints'] = data['Complaints'].apply(translate_text)  # Функцию перевода к колонке жалоб
data_translated['Keywords'] = data['Keywords'].apply(translate_text)  # Функцию перевода к колонке ключевых слов
data_translated['Answer'] = data['Answer'].apply(translate_text)  # Функцию перевода к колонке ответов

# Сохранение переведенного датасета в новый CSV файл
translated_file_path = 'dataset/Complaints_Answers_Translated.csv'  # Путь для сохранения нового файла
data_translated.to_csv(translated_file_path, index=False)  # Сохранить переведенный датасет в CSV файл без индексов

print(f'Translated dataset saved to {translated_file_path}')  # Сообщение о успешном сохранении файла

Translated dataset saved to dataset/Complaints_Answers_Translated.csv
